In [1]:
#imports 
import os

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import librosa
import soundfile
import re
import cv2
from sklearn.model_selection import KFold

import IPython.display as ipd

In [2]:
samples=[]
labels = []
gunshot_frequency_threshold = 0.25
sample_rate = 22050
sample_rate_per_two_seconds = 44100
input_shape = (sample_rate_per_two_seconds, 1)
base_dir = "/Users/laurenogden/Downloads/"
data_dir = base_dir + "REU_Samples_and_Labels/"
sound_data_dir = data_dir + "Samples/"


In [3]:
#read in csv file with labels
sound_types = pd.read_csv(data_dir + "labels.csv")


In [5]:
#read in all the wav files
#MULTICLASS CLASSIFICATION:
    #0: miscellanious
    #1: gunshot
    #2: fireworks
    #3: glassbreak
    
print("...Parsing sound data...")
sound_file_id = 0
sound_file_names = []

for file in os.listdir(sound_data_dir):
    if file.endswith(".wav"):
        try:
            #get label from the csv
            sound_file_id = int(re.search(r'\d+', file).group())
            sample, sample_rate = librosa.load(sound_data_dir + file)
            prescribed_label = sound_types.loc[sound_types["ID"] == sound_file_id, "Class"].values[0]
            print(file + "   " + prescribed_label)

            #for 2s or less long clips
            if len(sample) <= sample_rate_per_two_seconds:
                label = 0
                number_of_missing_hertz = sample_rate_per_two_seconds - len(sample)
                padded_sample = np.array(sample.tolist() + [0 for i in range(number_of_missing_hertz)])
                if prescribed_label == "gun_shot":
                    label = 1
                elif prescribed_label == "fireworks":
                    label = 2
                elif prescribed_label == "glassbreak":
                    label = 3

                #append the sample and label 
                samples.append(padded_sample)
                labels.append(label)
                sound_file_names.append(file)
            
            #for clips longer than 2s: split them up
            else:
                for i in range(0, sample.size - sample_rate_per_two_seconds, sample_rate_per_two_seconds):
                    sample_slice = sample[i : i + sample_rate_per_two_seconds]
                    label = 0
                    if prescribed_label == "gun_shot":
                        label = 1
                    elif prescribed_label == "fireworks":
                        label = 2
                    elif prescribed_label == "glassbreak":
                        label = 3
                    elif (np.max(abs(sample_slice)) < gunshot_frequency_threshold) and (prescribed_label == "gunshot"):
                        #check for silence to not mislabel gunshots
                        #maybe I should do this for fireworks and glassbreak also?
                        label = 0

                    #append the sample slice and label
                    samples.append(sample_slice)
                    labels.append(label)
                    sound_file_names.append(file)
        except:
            #if Librosa can't read a file or if something errs out
            print("Sound(s) not recognized by Librosa:" + file)
            pass


...Parsing sound data...
4666.wav   engine_idling
2217.wav   engine_idling
9342.wav   gun_shot
7409.wav   drilling
1078.wav   car_horn
6717.wav   children_playing
9424.wav   gun_shot
2571.wav   street_music
4100.wav   siren
6703.wav   drilling
9430.wav   gun_shot
9356.wav   gun_shot
8048.wav   engine_idling
4672.wav   street_music
1722.wav   drilling
8060.wav   children_playing
3135.wav   car_horn
5544.wav   children_playing
3653.wav   air_conditioner
8706.wav   siren
1044.wav   street_music
9418.wav   gun_shot
8712.wav   jackhammer
5236.wav   car_horn
3647.wav   children_playing
4128.wav   dog_bark
3121.wav   dog_bark
4896.wav   drilling
6059.wav   dog_bark
1736.wav   children_playing
8074.wav   car_horn
7347.wav   gun_shot
9381.wav   gun_shot
249.wav   siren
8921.wav   gun_shot
8935.wav   gun_shot
6918.wav   dog_bark
3860.wav   dog_bark
9395.wav   gun_shot
5587.wav   car_horn
275.wav   jackhammer
1087.wav   air_conditioner
513.wav   jackhammer
3848.wav   children_playing
507.wav   do

2558.wav   dog_bark
5237.wav   street_music
1051.wav   children_playing
8713.wav   dog_bark
7346.wav   drilling
8075.wav   engine_idling
1737.wav   children_playing
3120.wav   dog_bark
5545.wav   children_playing
4883.wav   car_horn
7352.wav   jackhammer
8061.wav   air_conditioner
9419.wav   gun_shot
1045.wav   engine_idling
8707.wav   gun_shot
3652.wav   gun_shot
5223.wav   jackhammer
9431.wav   gun_shot
2564.wav   engine_idling
8049.wav   car_horn
9357.wav   gun_shot
9343.wav   gun_shot
6070.wav   gun_shot
2216.wav   siren
3108.wav   children_playing
4667.wav   jackhammer
4101.wav   gun_shot
2570.wav   siren
9425.wav   gun_shot
6716.wav   jackhammer
1079.wav   engine_idling
7408.wav   children_playing
9355.wav   gun_shot
6066.wav   air_conditioner
1709.wav   dog_bark
7378.wav   drilling
2200.wav   air_conditioner
5209.wav   street_music
3678.wav   air_conditioner
4117.wav   air_conditioner
9433.wav   gun_shot
8739.wav   glassbreak
9427.wav   gun_shot
6714.wav   engine_idling
2572.wav

7392.wav   jackhammer
5585.wav   gun_shot
4843.wav   jackhammer
4857.wav   siren
263.wav   jackhammer
6098.wav   dog_bark
7386.wav   children_playing
6926.wav   children_playing
1091.wav   street_music
2598.wav   jackhammer
3686.wav   engine_idling
8923.wav   gun_shot
9383.wav   gun_shot
9397.wav   gun_shot
1913.wav   siren
8937.wav   gun_shot
6729.wav   drilling
8704.wav   siren
7437.wav   jackhammer
3889.wav   street_music
5546.wav   dog_bark
4880.wav   dog_bark
2229.wav   drilling
4658.wav   engine_idling
8062.wav   dog_bark
1720.wav   dog_bark
8076.wav   engine_idling
7345.wav   dog_bark
1734.wav   children_playing
9368.wav   gun_shot
4894.wav   dog_bark
3123.wav   dog_bark
3645.wav   drilling
5234.wav   siren
1052.wav   drilling
8710.wav   street_music
7423.wav   dog_bark
4102.wav   jackhammer
2573.wav   siren
6715.wav   jackhammer
9426.wav   gun_shot
8738.wav   glassbreak
288.wav   engine_idling
9340.wav   gun_shot
2201.wav   dog_bark
7379.wav   jackhammer
1708.wav   jackhammer
9

1916.wav   engine_idling
6089.wav   siren
272.wav   drilling
4846.wav   air_conditioner
3697.wav   jackhammer
6937.wav   children_playing
1080.wav   street_music
6923.wav   dog_bark
500.wav   jackhammer
3683.wav   drilling
4852.wav   air_conditioner
5594.wav   siren
7383.wav   children_playing
9345.wav   gun_shot
1719.wav   siren
2210.wav   dog_bark
4661.wav   street_music
3668.wav   car_horn
5219.wav   car_horn
2576.wav   dog_bark
6710.wav   street_music
9423.wav   gun_shot
9437.wav   gun_shot
8729.wav   air_conditioner
2204.wav   gun_shot
9351.wav   gun_shot
299.wav   children_playing
5543.wav   street_music
8067.wav   street_music
7354.wav   car_horn
9379.wav   gun_shot
1725.wav   drilling
1043.wav   air_conditioner
8701.wav   air_conditioner
7432.wav   street_music
3654.wav   engine_idling
5225.wav   dog_bark
3640.wav   children_playing
3898.wav   air_conditioner
6738.wav   children_playing
8715.wav   siren
7426.wav   engine_idling
8073.wav   air_conditioner
7340.wav   gun_shot
173

3864.wav   air_conditioner
8931.wav   gun_shot
6908.wav   dog_bark
8925.wav   gun_shot
3870.wav   dog_bark
4879.wav   engine_idling
9385.wav   gun_shot
1901.wav   street_music
4851.wav   air_conditioner
5597.wav   jackhammer
7380.wav   dog_bark
1929.wav   air_conditioner
6920.wav   children_playing
3680.wav   dog_bark
6934.wav   drilling
1083.wav   street_music
8919.wav   gun_shot
271.wav   drilling
5583.wav   jackhammer
4676.wav   children_playing
9352.wav   gun_shot
6061.wav   street_music
1068.wav   siren
9434.wav   gun_shot
7419.wav   drilling
4110.wav   siren
4104.wav   jackhammer
2575.wav   jackhammer
9420.wav   gun_shot
8058.wav   children_playing
9346.wav   gun_shot
4662.wav   jackhammer
7343.wav   engine_idling
5554.wav   siren
3125.wav   siren
1054.wav   children_playing
9408.wav   gun_shot
7425.wav   engine_idling
8716.wav   children_playing
1040.wav   children_playing
7431.wav   dog_bark
3657.wav   air_conditioner
4138.wav   engine_idling
5226.wav   jackhammer
2549.wav   st

...

7506.wav   engine_idling
8635.wav   children_playing
8621.wav   gun_shot
3012.wav   gun_shot
7274.wav   air_conditioner
9259.wav   gun_shot
1605.wav   jackhammer
391.wav   siren
2324.wav   dog_bark
4755.wav   drilling
9271.wav   gun_shot
6142.wav   engine_idling
9517.wav   gun_shot
8609.wav   air_conditioner
3984.wav   drilling
4027.wav   siren
2456.wav   drilling
9503.wav   gun_shot
6630.wav   dog_bark
9265.wav   gun_shot
6156.wav   siren
1639.wav   jackhammer
2330.wav   drilling
4741.wav   drilling
4972.wav   air_conditioner
346.wav   car_horn
8190.wav   siren
6817.wav   siren
352.wav   engine_idling
8184.wav   drilling
4966.wav   children_playing
6181.wav   jackhammer
1836.wav   engine_idling
4796.wav   dog_bark
5488.wav   children_playing
3947.wav   street_music
8812.wav   gun_shot
1188.wav   jackhammer
408.wav   engine_idling
8806.wav   gun_shot
1822.wav   jackhammer
2494.wav   engine_idling
8807.wav   gun_shot
6194.wav   jackhammer
4783.wav   engine_idling
5489.wav   siren
1837.w

6483.wav   jackhammer
2197.wav   street_music
5820.wav   drilling
7951.wav   children_playing
6497.wav   car_horn
2829.wav   dog_bark
7979.wav   street_music
8492.wav   drilling
2801.wav   air_conditioner
5808.wav   children_playing
136.wav   drilling
2815.wav   siren
8486.wav   siren
1893.wav   street_music
9217.wav   gun_shot
4733.wav   air_conditioner
4055.wav   dog_bark
2424.wav   siren
6642.wav   drilling
9571.wav   fireworks
6656.wav   gun_shot
9565.wav   fireworks
1139.wav   engine_idling
7548.wav   dog_bark
4041.wav   car_horn
2430.wav   siren
6130.wav   siren
9203.wav   gun_shot
5411.wav   street_music
3060.wav   dog_bark
8135.wav   engine_idling
7206.wav   engine_idling
1677.wav   drilling
6118.wav   jackhammer
485.wav   children_playing
8653.wav   drilling
7560.wav   jackhammer
4069.wav   air_conditioner
3706.wav   engine_idling
2418.wav   jackhammer
9559.wav   fireworks
1105.wav   car_horn
491.wav   jackhammer
7574.wav   drilling
7212.wav   drilling
1663.wav   dog_bark
5405

733.wav   dog_bark
2976.wav   drilling
874.wav   children_playing
1338.wav   street_music
6457.wav   jackhammer
6331.wav   engine_idling
9002.wav   gun_shot
4526.wav   air_conditioner
2157.wav   jackhammer
5638.wav   siren
4532.wav   siren
9016.wav   gun_shot
8308.wav   jackhammer
860.wav   engine_idling
7985.wav   engine_idling
20.wav   engine_idling
6443.wav   car_horn
2625.wav   jackhammer
4254.wav   drilling
8446.wav   air_conditioner
848.wav   car_horn
7775.wav   siren
690.wav   street_music
3275.wav   car_horn
1462.wav   street_music
7013.wav   air_conditioner
6319.wav   air_conditioner
8334.wav   children_playing
7007.wav   engine_idling
3261.wav   children_playing
5610.wav   dog_bark
5176.wav   car_horn
2619.wav   car_horn
3507.wav   dog_bark
7761.wav   air_conditioner
1310.wav   engine_idling
684.wav   drilling
7952.wav   dog_bark
2194.wav   car_horn
1489.wav   dog_bark
2180.wav   car_horn
5837.wav   children_playing
4297.wav   street_music
5189.wav   jackhammer
7946.wav   chi

2998.wav   gun_shot
4331.wav   drilling
2740.wav   children_playing
7638.wav   air_conditioner
905.wav   dog_bark
911.wav   dog_bark
2754.wav   dog_bark
4443.wav   jackhammer
5985.wav   drilling
8279.wav   siren
9167.wav   gun_shot
722.wav   children_playing
9198.wav   gun_shot
5952.wav   dog_bark
6283.wav   engine_idling
2783.wav   dog_bark
2797.wav   air_conditioner
3489.wav   drilling
7837.wav   dog_bark
6297.wav   siren
4480.wav   siren
3270.wav   drilling
5601.wav   engine_idling
1467.wav   engine_idling
6308.wav   jackhammer
8325.wav   jackhammer
7016.wav   siren
8443.wav   car_horn
7770.wav   jackhammer
695.wav   street_music
1301.wav   street_music
2608.wav   air_conditioner
4279.wav   drilling
3516.wav   drilling
5173.wav   air_conditioner
8457.wav   gun_shot
859.wav   street_music
19.wav   gun_shot
1473.wav   jackhammer
7002.wav   engine_idling
3264.wav   jackhammer
5615.wav   gun_shot
9007.wav   gun_shot
4523.wav   siren
2152.wav   street_music
2634.wav   engine_idling
871.w

8520.wav   jackhammer
1276.wav   gun_shot
7607.wav   jackhammer
8534.wav   engine_idling
3461.wav   children_playing
5010.wav   siren
2019.wav   siren
3307.wav   street_music
7161.wav   jackhammer
8252.wav   dog_bark
1510.wav   children_playing
2031.wav   air_conditioner
4440.wav   car_horn
7149.wav   children_playing
9164.wav   gun_shot
6531.wav   air_conditioner
912.wav   siren
3449.wav   air_conditioner
4326.wav   air_conditioner
5038.wav   jackhammer
2757.wav   street_music
906.wav   dog_bark
8508.wav   children_playing
9170.wav   gun_shot
6243.wav   jackhammer
2025.wav   jackhammer
4454.wav   children_playing
5992.wav   dog_bark
8285.wav   drilling
735.wav   jackhammer
721.wav   dog_bark
6294.wav   drilling
4483.wav   car_horn
2794.wav   siren
709.wav   drilling
1289.wav   children_playing
2780.wav   car_horn
5951.wav   jackhammer
1470.wav   drilling
7001.wav   siren
8332.wav   engine_idling
3267.wav   drilling
5616.wav   air_conditioner
2179.wav   drilling
3501.wav   siren
1316.w

In [6]:
#print number of samples available
print("The number of samples available for training is currently " + str(len(samples)) + '.')
print("The number of labels available for training is currently " + str(len(labels)) + '.')

The number of samples available for training is currently 7135.
The number of labels available for training is currently 7135.


In [7]:
#save the samples and labels as numpy arrays
np.save(base_dir + "gunshot_sound_samples_multiclass.npy", samples)
np.save(base_dir + "gunshot_sound_labels_multiclass.npy", labels)


# Data Augmentation

In [15]:
#data augmentation functions

def time_shift(wav):
    start_ = int(np.random.uniform(-wav.shape[0] * 0.5, wav.shape[0] * 0.5))
    if start_ >= 0:
        wav_time_shift = np.r_[wav[start_:], np.random.uniform(-0.001, 0.001, start_)]
    else:
        wav_time_shift = np.r_[np.random.uniform(-0.001, 0.001, -start_), wav[:start_]]
    return wav_time_shift
    
def change_pitch(wav, sample_rate):
    magnitude = int(np.random.uniform(-10, 10))
    wav_pitch_change = librosa.effects.pitch_shift(wav, sample_rate, magnitude)
    return wav_pitch_change
    
def speed_change(wav):
    speed_rate = np.random.uniform(0.7, 1.3)
    wav_speed_tune = cv2.resize(wav, (1, int(len(wav) * speed_rate))).squeeze()
    
    if len(wav_speed_tune) < len(wav):
        pad_len = len(wav) - len(wav_speed_tune)
        wav_speed_tune = np.r_[np.random.uniform(-0.001, 0.001, int(pad_len / 2)),
                               wav_speed_tune,
                               np.random.uniform(-0.001, 0.001, int(np.ceil(pad_len / 2)))]
    else: 
        cut_len = len(wav_speed_tune) - len(wav)
        wav_speed_tune = wav_speed_tune[int(cut_len / 2) : int(cut_len / 2) + len(wav)]
    return wav_speed_tune
    
def change_volume(wav, magnitude):
    # 0 < x < 1 quieter; x = 1 identity; x > 1 louder
    wav_volume_change = np.multiply(np.array([magnitude]), wav)
    return wav_volume_change
    
def add_background(wav, file, data_directory, label_to_avoid):
    label_csv = data_directory + "labels.csv"
    sound_directory = data_directory + "Samples/"
    sound_types = pd.read_csv(label_csv)
    bg_files = os.listdir(sound_directory)
    bg_files.remove(file)
    chosen_bg_file = bg_files[np.random.randint(len(bg_files))]
    jndex = int(chosen_bg_file.split('.')[0])
    while sound_types.loc[sound_types["ID"] == jndex, "Class"].values[0] == label_to_avoid:
        chosen_bg_file = bg_files[np.random.randint(len(bg_files))]
        jndex = int(chosen_bg_file.split('.')[0])
    bg, sr = librosa.load(sound_directory + chosen_bg_file)
    ceil = max((bg.shape[0] - wav.shape[0]), 1)
    start_ = np.random.randint(ceil)
    bg_slice = bg[start_ : start_ + wav.shape[0]]
    if bg_slice.shape[0] < wav.shape[0]:
        pad_len = wav.shape[0] - bg_slice.shape[0]
        bg_slice = np.r_[np.random.uniform(-0.001, 0.001, int(pad_len / 2)), bg_slice, np.random.uniform(-0.001, 0.001, int(np.ceil(pad_len / 2)))]
    wav_with_bg = wav * np.random.uniform(0.8, 1.2) + bg_slice * np.random.uniform(0, 0.5)
    return wav_with_bg

In [16]:
#augment the data

samples = np.array(samples)
labels = np.array(labels)
number_of_augmentations = 5
augmented_samples = np.zeros((samples.shape[0] * (number_of_augmentations + 1), samples.shape[1]))
augmented_labels = np.zeros((labels.shape[0] * (number_of_augmentations + 1),))
j = 0

for i in range (0, len(augmented_samples), (number_of_augmentations + 1)):
    file = sound_file_names[j]
    
    augmented_samples[i,:] = samples[j,:]
    augmented_samples[i + 1,:] = time_shift(samples[j,:])
    augmented_samples[i + 2,:] = change_pitch(samples[j,:], sample_rate)
    augmented_samples[i + 3,:] = speed_change(samples[j,:])
    augmented_samples[i + 4,:] = change_volume(samples[j,:], np.random.uniform())
    if labels[j] == 1:
        augmented_samples[i + 5,:] = add_background(samples[j,:], file, data_dir, "") 
    else:
        augmented_samples[i + 5,:] = add_background(samples[j,:], file, data_dir, "gun_shot")
    
    augmented_labels[i] = labels[j]
    augmented_labels[i + 1] = labels[j]
    augmented_labels[i + 2] = labels[j]
    augmented_labels[i + 3] = labels[j]
    augmented_labels[i + 4] = labels[j]
    augmented_labels[i + 5] = labels[j]
    j += 1

samples = augmented_samples
labels = augmented_labels

In [17]:
#print number of samples
print("The number of samples available for training is currently " + str(len(samples)) + '.')
print("The number of labels available for training is currently " + str(len(labels)) + '.')

The number of samples available for training is currently 42810.
The number of labels available for training is currently 42810.


In [18]:
#save the augmented
np.save(base_dir + "gunshot_augmented_sound_samples_multiclass.npy", samples)
np.save(base_dir + "gunshot_augmented_sound_labels_multiclass.npy", labels)